<a href="https://colab.research.google.com/github/rsekhar-vai/nlptc/blob/master/TextClassification_Start_Here.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!rm -r nlptc
!git clone https://github.com/rsekhar-vai/nlptc.git

rm: cannot remove 'nlptc': No such file or directory
Cloning into 'nlptc'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 46 (delta 16), reused 19 (delta 5), pack-reused 0
Unpacking objects: 100% (46/46), done.


In [0]:
from argparse import Namespace
from collections import Counter
import json
import os
import re
import string
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm_notebook
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import sys
import spacy
import torchtext
from torchtext import data
from torchtext.data import Field, BucketIterator


print('Python version:',sys.version)
print('Pandas version:',pd.__version__)
print('Pytorch version:', torch.__version__)
print('Torch Text version:', torchtext.__version__)
print('Spacy version:', spacy.__version__)


Python version: 3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0]
Pandas version: 1.0.3
Pytorch version: 1.4.0
Torch Text version: 0.3.1
Spacy version: 2.2.4


In [0]:
datadf = pd.read_csv('nlptc/data/train.csv')

encoder = preprocessing.LabelEncoder()
datadf['sentiment'] = encoder.fit_transform(datadf['sentiment'])

traindf, testdf = train_test_split(datadf)
traindf, valdf = train_test_split(traindf)
traindf.to_csv("traindf.csv", index=False)
testdf.to_csv("testdf.csv", index=False)
valdf.to_csv("valdf.csv", index=False)
traindf.head()


,textID,text,selected_text,sentiment
13545,ff2b9c53ac,You have a DM amigui,You have a DM amigui,1
2833,eda6389e34,My Power was cut off thats where i went,My Power was cut off,0
1028,ee0d5790a6,it`s Mothers Day today,it`s Mothers Day today,1
9782,c797e813a3,It is sad... I feel bad when I hear it. They ...,feel bad,0
1508,48377c4bef,"praying to be humble is always dangerous, He ...","praying to be humble is always dangerous, He w...",1


In [0]:
def clean_text(text):
    text = re.sub(r'[^A-Za-z0-9]+', ' ', text) # remove non alphanumeric character
    return text.strip()
nlp = spacy.load('en_core_web_sm')
def tokenizer(sentence):
    return [w.text.lower() for w in nlp(clean_text(sentence))]

TEXT = data.Field(tokenize = tokenizer)
LABEL = data.LabelField()
fields = [(None,None),('text',TEXT),(None,None),('label',LABEL)]


In [0]:
 train_data, valid_data, test_data = data.TabularDataset.splits(
                                         path = '',
                                         train = 'traindf.csv',
                                         validation = 'valdf.csv',
                                         test = 'testdf.csv',
                                         format = 'csv',
                                         fields = fields,
                                         skip_header = True
     )
print(vars(train_data[0]))


KeyboardInterrupt: ignored

In [0]:
 %%time
MAX_VOCAB_SIZE = 25000
#vec = vocab.Vectors('glove.6B.100d.txt', 'D:/qBots/nlptc/glove_embedding/')
TEXT.build_vocab(train_data, valid_data,
                  max_size = MAX_VOCAB_SIZE, 
                  vectors = "glove.6B.100d", 
#                vectors = vec,
                  unk_init = torch.Tensor.normal_)
LABEL.build_vocab(train_data)


In [0]:
TEXT.vocab.vectors.shape
TEXT.vocab.vectors[TEXT.vocab.stoi['testing']]
vars(train_data[-1])
print(LABEL.vocab.stoi)

In [0]:
BATCH_SIZE = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_key=lambda x: len(x.text), 
    sort_within_batch=True, 
    device = device)

len(train_data), len(valid_data)


NameError: ignored

In [0]:
batch = next(iter(train_data))

batch.__dict__

batch.label

len(TEXT.vocab), len(LABEL.vocab)


NameError: ignored

In [0]:
model = VDCNN(len(LABEL.vocab),EMBEDDING_DIM,K_MAX,len(TEXT.vocab),
              DROPOUT, PAD_IDX).to(args.device)


In [0]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')


In [0]:
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)


In [0]:
def init_weights(m: nn.Module):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)


In [0]:
model.apply(init_weights)
optimizer = optim.Adam(model.parameters(),lr=args.learning_rate)
loss_func = nn.CrossEntropyLoss()
model = model.to(device)
loss_func = loss_func.to(device)


In [0]:
def categorical_accuracy(preds, y):
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]])



In [0]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0    
    model.train()    
    for batch in iterator:        
        optimizer.zero_grad()        
        predictions = model(batch.text)        
        loss = criterion(predictions, batch.label)        
        acc = categorical_accuracy(predictions, batch.label)        
        loss.backward()        
        optimizer.step()        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)



In [0]:
def evaluate(model, iterator, criterion):    
    epoch_loss = 0
    epoch_acc = 0    
    model.eval()    
    with torch.no_grad():    
        for batch in iterator:
            predictions = model(batch.text)            
            loss = criterion(predictions, batch.label)            
            acc = categorical_accuracy(predictions, batch.label)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)



In [0]:
def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


In [0]:
import time
N_EPOCHS = 5
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss, train_acc = train(model, train_iterator, optimizer, loss_func)
    valid_loss, valid_acc = evaluate(model, valid_iterator, loss_func)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut5-model.pt')
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

test_loss = evaluate(model, test_iterator, criterion)
print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

